<h1>Welcome to my TxMM project</h1> This project is about the missing of Hebe and Sanne in October 2022 in the Netherlands.

By: Roel Duijsings

Install needed packages:

In [9]:
#! pip install datasets transformers

<h2>Load the training data</h2>

<h3>Rewrite training data: replace tweet_id by tweet_text</h3>
Create a training_data_{limit}.csv file

In [19]:
from snscrape.modules.twitter import TwitterTweetScraper
import pandas as pd
from preprocess import replacePlaceholders

limit = 10
tweets = []
with open("EmotioNL_tweets.txt") as f:
    next(f)  # skip header
    i = 0
    # For every tweet in the file...
    for line in f:
        if i >= limit:
            break

        line_splitted = line.split()
        tweet_id = line_splitted[1][2:-2]
        category = line_splitted[-1]

        try:
            # ...scrape that tweet from Twitter using tweet_id
            for tweet in TwitterTweetScraper(tweet_id).get_items():
                text = replacePlaceholders(tweet.content)
                # TODO: how to correctly save the data?
                data_point = {"label": category, "text": text}
                tweets.append(data_point)
                # print(tweet_id, data_point)
        except:
            print("Something went wrong when scraping the tweets.")
            pass
        i += 1

df = pd.DataFrame(tweets)
df.index.name = "idx"
df.to_csv(f"training_data_{len(df.index)}.csv")  # save as .csv file
print("- - - Finished loading training data - - -")


- - - Finished loading training data - - -


<h3> Train the model on the training data</h3>

In [20]:
from tabnanny import check
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "pdelobelle/robbert-v2-dutch-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)


loading configuration file config.json from cache at C:\Users\roell/.cache\huggingface\hub\models--pdelobelle--robbert-v2-dutch-base\snapshots\94c743437a36425637fcf29d3c939c813da0398c\config.json
Model config RobertaConfig {
  "_name_or_path": "pdelobelle/robbert-v2-dutch-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 40000
}

loading file vocab.json from cache at C:\Users\roell/.

In [21]:
from datasets.load import load_dataset


def tokenize_function(tweet):
    return tokenizer(tweet["text"], padding="max_length", truncation=True)


data_file = "training_data_10.csv"
train_dataset = load_dataset("csv", data_files=data_file)
train_dataset_tokenized = train_dataset.map(tokenize_function, batched=True)


Using custom data configuration default-5cc789a14ef7e4ad


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 332.78it/s]
c:\Users\roell\Documents\Master AI\Text and Multimedia Mining v2\Research paper\TxMM-research-project\.venv\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to C:/Users/roell/.cache/huggingface/datasets/csv/default-5cc789a14ef7e4ad/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 105.09ba/s]


In [24]:
train_dataset["train"][0:3]


{'idx': [0, 1, 2],
 'label': ['anger', 'anger', 'anger'],
 'text': ['@USER Maandagmorgen en extra druk😣Dat is nu voordeel van niet meer werken op maandag al dit nieuws lezen!',
  '@USER Echt...zweet en shag..en dat een hele dag😡Je zal ermee op kantoor zitten😝',
  '@USER @USER @USER D66Pechtold zei 1e BELANGRIJKSTE!poging😂dream on Pechtold gelul id ruimte! PVV id 2e grtste!wat e kleuterklasje.pesten uitsluiten z nie 2017']}

In [23]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized["train"],  # TODO: deze dataset komt niet goed door! check format!
    tokenizer=tokenizer,
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
  0%|          | 0/6 [25:51<?, ?it/s]
The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text, idx. If text, idx are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\roell\Documents\Master AI\Text and Multimedia Mining v2\Research paper\TxMM-research-project\.venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`label` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

<h2>Load the testdata</h2>

In [2]:
from preprocess import preprocess
query = "(hebe) lang:nl until:2022-10-21 since:2022-10-17"
maxTweets = 100000
data = preprocess(query, maxTweets)

In [3]:
data.to_csv(f"tweets_{len(data.index)}.csv")